In [1]:
import pandas as pd
import sqlalchemy
import geopandas as gpd
import seaborn as sns
from shapely import wkt
from itertools import cycle, islice

In [2]:
df = pd.read_csv(r'..\..\..\outputs\network\network_results.csv')

# Congested network components by time of day
df.columns

# Get freeflow from 20to5 period

# Exclude trips taken on non-designated facilities (facility_type == 0)
# These are artificial (weave lanes to connect HOV) or for non-auto uses 
df = df[df['data3'] != 0]    # data3 represents facility_type

# calculate total link VMT and VHT
df['VMT'] = df['@tveh']*df['length']
df['VHT'] = df['@tveh']*df['auto_time']/60

# Define facility type
df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

# Calculate delay
# Select links from overnight time of day
delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

# Merge delay field back onto network link df
df = pd.merge(df, delay_df, on='ij', how='left')

# Calcualte hourly delay
df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

# Calcualte speed as a percent of freeflow
df['flow_index'] = df['auto_time']/df['freeflow_time']

df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})

df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,1], labels=['Severe','Heavy','Moderate','Light'])

In [3]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf


In [4]:
def format_df(df, col_list, col_name, index_list, index_name):
    
    # Re-order columns
    df = df[col_list]
    # Re-order index
    df = df.reindex(index_list)
    # Set visible column and index names
    df.columns.name = col_name
    df.index.name = index_name
    
    return df


In [5]:
# set chart style
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, 
                rc={"lines.linewidth": 1.5})
colors = ['#91268F','#F05A28','#8CC63E','#00A7A0','#4C4C4C', '#BFE9E7']
sns.set(font="Pragmatica Condensed Light")

In [6]:
# Load regional geographies layer to intersect with network shapefile
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'cities', version, crs=crs, is_table=False)

# Intersect geography this with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
gdf_intersect = gpd.overlay(gdf_network, gdf_shp, how="union", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

### IMPORTANT
# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

C:\Users\modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df = df[['@tveh','congestion_category','i_node','j_node','tod','total_delay']].merge(gdf_intersect, 
                                                             left_on=['i_node','j_node'],
                                                            right_on=['i','j'],
                                                                  how='left')
df['VMT'] = df['new_length']*df['@tveh']

# Assume speed is constant or assigned link
# When links are split, we take proportionate delay by length

# calculate total delay
df['delay'] = (df['new_length']/df['length'])*df['total_delay']
# Note that we won't get all of the data because some links are outside the region boundaries
# Optionally, scale these numbers by the total delay field total_delay for consistency
df['scaled_delay'] = df['delay']*(df['total_delay'].sum()/df['delay'].sum())

# Congestion by City
### Total VMT & Percent of VMT by Congestion Level

- **Severe: < 25% Speed Limit** (e.g., less than 15 mph on a road with speed limit of 60 mph)
- **Heavy: 25-50% Speed Limit** (e.g., 15-30 mph on a road with speed limit of 60 mph)
- **Moderate: 50-70% Speed Limit** (e.g, 30-42 mph on a road with speed limit of 60 mph)

#### All Day

In [8]:
pd.options.display.max_rows = None

In [9]:
df.rename(columns={'city_name': 'City'}, inplace=True)
df['City'] = df['City'].fillna('Not in City Limits')

# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,32,69,"30,178","149,774"
Arlington,218,"1,653","14,287","311,187"
Auburn,"3,981","58,799","139,376","1,631,433"
Bainbridge Island,21,239,"1,203","213,876"
Beaux Arts,0,0,0,404
Bellevue,"18,931","206,114","449,596","3,087,704"
Black Diamond,139,683,"1,131","47,986"
Bonney Lake,"1,808","14,967","32,820","207,734"
Bothell,"14,881","118,543","236,972","1,326,971"


In [10]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.0%,0.0%,16.8%,83.2%
Arlington,0.1%,0.5%,4.4%,95.1%
Auburn,0.2%,3.2%,7.6%,89.0%
Bainbridge Island,0.0%,0.1%,0.6%,99.3%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,0.5%,5.5%,11.9%,82.1%
Black Diamond,0.3%,1.4%,2.3%,96.1%
Bonney Lake,0.7%,5.8%,12.8%,80.7%
Bothell,0.9%,7.0%,14.0%,78.2%


#### AM Peak (7 to 8 AM)

In [11]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0,2,"5,522","6,923"
Arlington,18,56,"1,082","22,008"
Auburn,365,"9,608","25,048","99,600"
Bainbridge Island,2,18,131,"16,060"
Beaux Arts,0,0,0,31
Bellevue,"1,299","36,860","59,331","185,850"
Black Diamond,22,68,254,"3,489"
Bonney Lake,325,"1,168","3,425","12,498"
Bothell,684,"25,588","24,720","65,726"


In [12]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.0%,0.0%,44.4%,55.6%
Arlington,0.1%,0.2%,4.7%,95.0%
Auburn,0.3%,7.1%,18.6%,74.0%
Bainbridge Island,0.0%,0.1%,0.8%,99.1%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,0.5%,13.0%,20.9%,65.6%
Black Diamond,0.6%,1.8%,6.6%,91.0%
Bonney Lake,1.9%,6.7%,19.7%,71.8%
Bothell,0.6%,21.9%,21.2%,56.3%


#### PM Peak (5 to 6 PM)

In [13]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,17,1,"5,746","7,765"
Arlington,33,185,"1,758","24,800"
Auburn,361,"10,809","14,474","127,112"
Bainbridge Island,1,25,79,"18,676"
Beaux Arts,0,0,0,30
Bellevue,"4,366","28,564","61,614","213,242"
Black Diamond,23,50,82,"4,155"
Bonney Lake,148,"2,119","3,998","14,382"
Bothell,"1,709","18,326","39,726","73,878"


In [14]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.1%,0.0%,42.5%,57.4%
Arlington,0.1%,0.7%,6.6%,92.6%
Auburn,0.2%,7.1%,9.5%,83.2%
Bainbridge Island,0.0%,0.1%,0.4%,99.4%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,1.4%,9.3%,20.0%,69.3%
Black Diamond,0.5%,1.2%,1.9%,96.4%
Bonney Lake,0.7%,10.3%,19.4%,69.7%
Bothell,1.3%,13.7%,29.7%,55.3%


# Network Delay
Totals scaled to match network totals. 

In [15]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('City').sum()[['scaled_delay']]
_df.loc['Total',:] = _df.sum() 
_df

,scaled_delay
City,
Algona,674
Arlington,321
Auburn,"4,799"
Bainbridge Island,58
Beaux Arts,0
Bellevue,"15,743"
Black Diamond,32
Bonney Lake,"1,067"
Bothell,"10,158"


Totals unscaled. Some links cross outside the region and delay on these links are excluded at boundaries

In [16]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('City').sum()[['delay']]
_df.loc['Total',:] = _df.sum() 
_df

,delay
City,
Algona,576
Arlington,275
Auburn,"4,106"
Bainbridge Island,49
Beaux Arts,0
Bellevue,"13,471"
Black Diamond,27
Bonney Lake,913
Bothell,"8,692"
